In [2]:
import sys, os, time

import open3d as o3d
import trimesh
import openmesh as om

import cv2

import scipy as sp
from scipy.sparse.linalg import eigs
from scipy.sparse import csr_matrix, coo_matrix, identity, vstack
from scipy.sparse.linalg import spsolve, lsqr
from scipy.spatial import Delaunay, Voronoi

import numpy as np

import matplotlib.pyplot as plt

from tqdm import tqdm

%matplotlib inline
# Tests of Openmesh
# https://gitlab.vci.rwth-aachen.de:9000/OpenMesh/openmesh-python/tree/master/tests


In [3]:

def read_mesh_om(path_in_models):
    return om.read_trimesh(os.path.join('..', 'models', path_in_models))
def write_mesh_om(mesh, path_in_models):
    om.write_mesh(os.path.join('..', 'models', path_in_models), mesh, vertex_color=True)
def show_mesh_o3d(plys):
    o3d.visualization.draw_geometries(plys)
def write_mesh_o3d(path, mesh):
    o3d.io.write_triangle_mesh(path, mesh)
def read_mesh_o3d(mesh_fp):
    return o3d.io.read_triangle_mesh(mesh_fp)
def read_mesh_trimesh(path_in_models):
    return trimesh.load(os.path.join('..', 'models', path_in_models))
def get_vertices_from_trimesh(mesh):
    return np.asarray(mesh.vertices)
def get_vertice_normals_from_trimesh(mesh):
    return np.asarray(mesh.vertex_normals)


In [4]:

### Calculate Voronoi Poles
INFINITY = 1e9
EPS = 1e-9
def calculate_voronoi_poles(mesh):
    
    vertices = get_vertices_from_trimesh(mesh)
    vertice_normals = get_vertice_normals_from_trimesh(mesh)

    print("Calculating Voronoi Diagram.")
    vor = Voronoi(vertices)
    vor_centers = vor.vertices
    cells = vor.regions
    cell_indices = vor.point_region

    vertices_total = vertices.shape[0]
    voronoi_poles = np.zeros(vertices.shape)
    print("Calculating the Voronoi Pole for each vertex.")
    for vi in tqdm(range(vertices_total)):
        vor_cell = cells[cell_indices[vi]]
        vertice = vertices[vi]
        vertice_normal = vertice_normals[vi]
        max_neg_proj = INFINITY
        voronoi_pole = None
        for vci in vor_cell:
            vor_center = vor_centers[vci]
            if vci == -1:
                continue
            proj = np.dot(vor_center - vertice, vertice_normal)
            if proj < max_neg_proj:
                max_neg_proj = proj
                voronoi_pole = vor_center
        voronoi_poles[vi] = voronoi_pole

    return voronoi_poles


In [50]:

### Calculate Cotangent Laplace Operator
def trimesh_Generate_Area_List(mesh):

    area_list = np.zeros((len(mesh.vertices)))
    face_count = mesh.faces.shape[0]
    face_area = np.asarray(mesh.area_faces)
    faces = np.asarray(mesh.faces)
    for i in range(face_count):
        area_list[faces[i]] += face_area[i] / 3
    
    area_list /= np.mean(area_list)
    return area_list


def trimesh_Generate_Laplace_matrix(mesh):
    
    print('Constructing Laplace Matrix.')
    
    vertices_face_indexs = [[0,1,2],[1,0,2],[2,0,1]] 
    laplace_dict = {}
    area_list = trimesh_Generate_Area_List(mesh)
    face_angles = np.asarray(mesh.face_angles)
    faces = np.asarray(mesh.faces)
    face_count = faces.shape[0]
    
    # with tqdm(total=face_count) as tbar:
    for face, angles in zip(faces, face_angles):
        for i in range(3):
            current_angle = angles[i]
            v0_index, v1_index, v2_index = face[vertices_face_indexs[i]]
            delta = 0.5 / area_list[v0_index] / np.tan(current_angle)
            if delta > EPS:
                laplace_dict[(v1_index, v1_index)] = laplace_dict.get((v1_index, v1_index), 0) - delta
                laplace_dict[(v2_index, v2_index)] = laplace_dict.get((v2_index, v2_index), 0) - delta
                laplace_dict[(v1_index, v2_index)] = laplace_dict.get((v1_index, v2_index), 0) + delta
                laplace_dict[(v2_index, v1_index)] = laplace_dict.get((v2_index, v1_index), 0) + delta
        # tbar.update(1)

    # Construct CSR Matrix
    rows, cols = zip(*laplace_dict.keys())
    values = list(laplace_dict.values())
    coo = coo_matrix((values, (rows, cols)), shape=(len(mesh.vertices), len(mesh.vertices)))
    csr = coo.tocsr()

    return csr


In [6]:

### Solve Equation
def construct_Equation(wL, wH, wM, vertices, L, voronoi_poles):
    vn = vertices.shape[0]
    WH_L = wH * identity(vn)
    WM_L = wM * identity(vn)
    A = vstack((wL * L, WH_L, WM_L))
    b = np.vstack((np.zeros((vn, 3)), wH * vertices, wM * voronoi_poles))
    return A, b


In [15]:
flatten = lambda l: [item for sublist in l for item in sublist]

def laplacian_smoothing(mesh, smooth_factor=1.0, iterations=1):
   
    for _ in range(iterations):
        n = mesh.vertices.shape[0]
        vertex_degs = [len(neighbors) for neighbors in mesh.vertex_neighbors]
        sparse_mat_rows = [[row_idx] * (deg + 1) for row_idx, deg in enumerate(vertex_degs)]
        sparse_mat_cols = [neighbors + [row_idx] for row_idx, neighbors in enumerate(mesh.vertex_neighbors)]

        
        sparse_mat_weights = [[-1.0] * deg + [deg * 1.0] for deg in vertex_degs]   

        sparse_mat_rows = flatten(sparse_mat_rows)
        sparse_mat_cols = flatten(sparse_mat_cols) 

        sparse_mat_weights = [float(u) / row_weights[-1]
                                    for row_weights in sparse_mat_weights
                                    for u in row_weights]
        L =coo_matrix((sparse_mat_weights, (sparse_mat_rows, sparse_mat_cols)), shape=(n , n))
        L = L.tocsr()  # Compress the sparse matrix..
        v = np.array(mesh.vertices)
        diff = L * v
        mesh.vertices = v - smooth_factor * diff

In [45]:
name = 'armadillo'
mesh = read_mesh_trimesh(name+ '.obj')
laplacian_smoothing(mesh,smooth_factor = 1.0)
write_mesh_o3d('../models/result/test_smooth.obj', mesh.as_open3d)

In [51]:
### Iteration
it = 10
wL = 0.001
wH = 0.1
wM = 0.2
name = 'armadillo'
new_mesh = name + '_0_smooth.obj'
# new_mesh = 'asculpt.obj'
mesh = read_mesh_trimesh(name+'.obj')
voronoi_poles = calculate_voronoi_poles(mesh)

for i in range(it):

    laplace_csr = trimesh_Generate_Laplace_matrix(mesh)

    vertices = get_vertices_from_trimesh(mesh)
    A, b = construct_Equation(wL, wH, wM, vertices, laplace_csr, voronoi_poles)

    new_vertices0 = lsqr(A, b[:, 0])
    new_vertices1 = lsqr(A, b[:, 1])
    new_vertices2 = lsqr(A, b[:, 2])

    new_vertices = np.zeros(vertices.shape)
    new_vertices[:, 0] = new_vertices0[0]
    new_vertices[:, 1] = new_vertices1[0]
    new_vertices[:, 2] = new_vertices2[0]

    mesh.vertices = new_vertices
    
    save_index = int(i/5)
    write_mesh_o3d('../models/result/' + new_mesh, mesh.as_open3d)
    mesh = read_mesh_trimesh('result/' + new_mesh)
    laplacian_smoothing(mesh,smooth_factor = 1.0)
    new_mesh = name + '_%d_smooth.obj' % (save_index + 1)
    smoothed = '%d_smooth.obj' % (save_index + 1)
    write_mesh_o3d('../models/result/' + smoothed, mesh.as_open3d)
    
    


Calculating Voronoi Diagram.
Calculating the Voronoi Pole for each vertex.


100%|██████████| 25193/25193 [00:01<00:00, 20856.93it/s]


Constructing Laplace Matrix.
Constructing Laplace Matrix.
Constructing Laplace Matrix.
Constructing Laplace Matrix.
Constructing Laplace Matrix.
Constructing Laplace Matrix.
Constructing Laplace Matrix.
Constructing Laplace Matrix.
Constructing Laplace Matrix.
Constructing Laplace Matrix.


In [ ]:
# test-1

In [ ]:
import sys, os, time

import open3d as o3d
import trimesh
import openmesh as om
import pyrender
import numpy as np
from scipy.sparse import coo_matrix, csr_matrix



# import cv2

import scipy as sp
from scipy.sparse.linalg import eigs
from scipy.sparse import csr_matrix, identity
from scipy.sparse.linalg import spsolve

import numpy as np

import matplotlib.pyplot as plt

from tqdm import tqdm

%matplotlib inline

def read_mesh_om(path_in_models):
    return om.read_trimesh(os.path.join('..', 'models', path_in_models))
def write_mesh_om(mesh, path_in_models):
    om.write_mesh(os.path.join('..', 'models', path_in_models), mesh, vertex_color=True)
def show_mesh_o3d(plys):
    o3d.visualization.draw_geometries(plys)
def write_mesh_o3d(path, mesh):
    o3d.io.write_triangle_mesh(path, mesh)
def read_mesh_o3d(mesh_fp):
    return o3d.io.read_triangle_mesh(mesh_fp)

In [12]:
help(om.write_mesh)

Help on built-in function write_mesh in module openmesh:

write_mesh(...) method of builtins.PyCapsule instance
    write_mesh(*args, **kwargs)
    Overloaded function.
    
    1. write_mesh(filename: str, mesh: openmesh.TriMesh, binary: bool = False, msb: bool = False, lsb: bool = False, swap: bool = False, vertex_normal: bool = False, vertex_color: bool = False, vertex_tex_coord: bool = False, halfedge_tex_coord: bool = False, edge_color: bool = False, face_normal: bool = False, face_color: bool = False, color_alpha: bool = False, color_float: bool = False, status: bool = False, texture_file: str = '', material_file_extension: str = '.mat') -> None
    
    2. write_mesh(filename: str, mesh: openmesh.PolyMesh, binary: bool = False, msb: bool = False, lsb: bool = False, swap: bool = False, vertex_normal: bool = False, vertex_color: bool = False, vertex_tex_coord: bool = False, halfedge_tex_coord: bool = False, edge_color: bool = False, face_normal: bool = False, face_color: bool = Fa

In [13]:
# print(mesh.vertices())
# set = False
# for v_it in mesh.vertices():
#     if not set:
#         mesh.set_point(v_it, np.array([-1, -1 , 1]))
#         set = True
#     print(mesh.point(v_it), type(mesh.point(v_it)))
#     mesh.set_color(v_it, np.array([255.,182.,193.,255.])/255.)

# # for f_it in mesh.faces():
# #     mesh.set_color(f_it, np.array([255.,182.,193.,255.])/255.)
# # for c_it in mesh.face_colors():
# #     print(c_it, type(c_it))
# write_mesh_om(mesh, 'test/cube-copy.off')

In [14]:
# write_mesh_om(mesh, 'test\cube-copy.off')

In [15]:
# # def run_gui(scene):        
# #     v = pyrender.Viewer(scene, use_raymond_lighting=True)
# #     del v

# # def PyView(FilePath):
# #     scene = pyrender.Scene(ambient_light=0.4*np.array([1.0, 1.0, 1.0, 1.0]))
# #     mesh_bunny = os.path.join(FilePath)
# #     m2 = trimesh.load(mesh_bunny)
# #     mesh_node = scene.add(pyrender.Mesh.from_trimesh(m2))
# #     run_gui(scene)   
# TestPath = '../models/test/cube-copy.off'
# PyView(TestPath)

In [16]:
# FaceIndex = 0
# for f_it in mesh.faces():
#     # help(f_it)
#     for v_it in om.FaceVertexIter(mesh, f_it):
#         print(mesh.point(v_it))

#         mesh.set_color(v_it, np.array([250.,250.,200.,200.])/255.)
#     FaceIndex +=1
#     # break
#     print('--------',FaceIndex)
# write_mesh_om(mesh, 'test\cube-copy.off')

# # 12 faces

In [17]:
def get_Vertices_From_Face(mesh, f_it):
    vertices = np.zeros((3,3))
    vertices_index = np.zeros((3),dtype=int)
    for index, v_it in enumerate(om.FaceVertexIter(mesh, f_it)):
        vertices[index] = mesh.point(v_it)
        vertices_index[index] = int(v_it.idx())
        # print(v_it.idx())
    return vertices_index, vertices
    # print(vertices)

    


In [18]:
def calculate_Triangle_Area(a, b, c):
    # abc are the position of 3 vertices.
    ba = b - a
    ca = c - a
    area = np.linalg.norm(np.cross(ba, ca)) / 2.
    return area

def calculate_Triangle_Angle(a,b,c):
    # get angle for a.
    ba = b - a
    ca = c - a
    cosi = ba @ ca / (np.linalg.norm(ba) * np.linalg.norm(ca))
    angle = np.arccos(cosi)
    return angle


In [19]:
# # For each face
# #   get vertices
# #   get area for the face
# #   assign 1/3 area to each vertx, get area list
# # For each face
# #   get vertices
# #   get delta
# #   adjust corresponding value in laplace matrix
 
# mesh = read_mesh_om('cube.obj')

# # Pre-settings

# vertices_face_indexs = [[0,1,2],[1,0,2],[2,0,1]] 
# laplace_matrix = np.zeros((mesh.n_vertices(), \
#                            mesh.n_vertices()), dtype=float)
# area_list = np.zeros((mesh.n_vertices())) # the area for each vertex
# FaceIndex = 0
# print(area_list.shape)


# for f_it in mesh.faces():
#     # help(f_it)
#     vertices_index, vertices = get_Vertices_From_Face(mesh,f_it)
#     current_area = calculate_Triangle_Area(vertices[0],vertices[1],vertices[2])
#     # print(current_area);
#     area_list[vertices_index] += current_area/3

#     # return list

# for f_it in mesh.faces():
#     vertices_index, vertices = get_Vertices_From_Face(mesh,f_it)
#     for current_indexs in vertices_face_indexs:
        
#         v0, v1, v2 = vertices[current_indexs]
#         v0_index, v1_index,v2_index = vertices_index[current_indexs]

#         current_angle = calculate_Triangle_Angle(v0, v1, v2)

#         delta = 0.5 / area_list[v0_index] / np.tan(current_angle)
#         # print(v1_index,v2_index,delta)
#         laplace_matrix[v1_index,v1_index] -= delta
#         laplace_matrix[v2_index,v2_index] -= delta

#         laplace_matrix[v1_index,v2_index] += delta        
#         laplace_matrix[v2_index,v1_index] += delta
#     FaceIndex +=1
#     print('--------',FaceIndex)
    

In [20]:
# For each face
#   get vertices
#   get area for the face
#   assign 1/3 area to each vertx, get area list
# For each face
#   get vertices
#   get delta
#   adjust corresponding value in laplace matrix
 
mesh = read_mesh_om('cube.obj')

def generate_Area_List(mesh):
    
    area_list = np.zeros((mesh.n_vertices()))
    for f_it in mesh.faces():
    # help(f_it)
        vertices_index, vertices = get_Vertices_From_Face(mesh,f_it)
        current_area = calculate_Triangle_Area(vertices[0],vertices[1],vertices[2])
        # print(current_area);
        area_list[vertices_index] += current_area/3
    return area_list
def generate_Laplace_matrix(mesh):
    vertices_face_indexs = [[0,1,2],[1,0,2],[2,0,1]] 
    laplace_matrix = np.zeros((mesh.n_vertices(), \
                               mesh.n_vertices()), dtype=float)
    FaceIndex = 0
    area_list = generate_Area_List(mesh)
    for f_it in mesh.faces():
        vertices_index, vertices = get_Vertices_From_Face(mesh,f_it)
        for current_indexs in vertices_face_indexs:
            
            v0, v1, v2 = vertices[current_indexs]
            v0_index, v1_index,v2_index = vertices_index[current_indexs]

            current_angle = calculate_Triangle_Angle(v0, v1, v2)

            delta = 0.5 / area_list[v0_index] / np.tan(current_angle)
            if delta > 0.000001:
            # print(v1_index,v2_index,delta)
                laplace_matrix[v1_index,v1_index] -= delta
                laplace_matrix[v2_index,v2_index] -= delta

                laplace_matrix[v1_index,v2_index] += delta        
                laplace_matrix[v2_index,v1_index] += delta

        FaceIndex +=1
        # print('--------',FaceIndex)
    return laplace_matrix

laplace_matrix = generate_Laplace_matrix(mesh)
print(laplace_matrix)

[[-3.9   1.35  1.35  0.    0.    0.    1.2   0.  ]
 [ 1.35 -3.9   0.    1.35  0.    0.    0.    1.2 ]
 [ 1.35  0.   -3.9   1.35  1.2   0.    0.    0.  ]
 [ 0.    1.35  1.35 -3.9   0.    1.2   0.    0.  ]
 [ 0.    0.    1.2   0.   -3.9   1.35  1.35  0.  ]
 [ 0.    0.    0.    1.2   1.35 -3.9   0.    1.35]
 [ 1.2   0.    0.    0.    1.35  0.   -3.9   1.35]
 [ 0.    1.2   0.    0.    0.    1.35  1.35 -3.9 ]]


In [21]:
# For each face
#   get vertices
#   get area for the face
#   assign 1/3 area to each vertx, get area list
# For each face
#   get vertices
#   get delta
#   adjust corresponding value in laplace matrix
 
mesh = read_mesh_om('cube.obj')

def generate_Area_List(mesh):
    
    area_list = np.zeros((mesh.n_vertices()))
    for f_it in mesh.faces():
    # help(f_it)
        vertices_index, vertices = get_Vertices_From_Face(mesh,f_it)
        current_area = calculate_Triangle_Area(vertices[0],vertices[1],vertices[2])
        # print(current_area);
        area_list[vertices_index] += current_area/3
    return area_list


def generate_Laplace_matrix(mesh):
    vertices_face_indexs = [[0,1,2],[1,0,2],[2,0,1]] 
    laplace_dict = {}
    area_list = generate_Area_List(mesh)
    for f_it in mesh.faces():
        vertices_index, vertices = get_Vertices_From_Face(mesh,f_it)
        for current_indexs in vertices_face_indexs:
            
            v0, v1, v2 = vertices[current_indexs]
            v0_index, v1_index,v2_index = vertices_index[current_indexs]

            current_angle = calculate_Triangle_Angle(v0, v1, v2)

            delta = 0.5 / area_list[v0_index] / np.tan(current_angle)
            if delta > 0.000001:

                laplace_dict[(v1_index,v1_index)] = laplace_dict.get(((v1_index,v1_index)),0) - delta
                laplace_dict[(v2_index,v2_index)] = laplace_dict.get(((v2_index,v2_index)),0) - delta

                laplace_dict[(v1_index,v2_index)] = laplace_dict.get(((v1_index,v2_index)),0) + delta
                laplace_dict[(v2_index,v1_index)] = laplace_dict.get(((v2_index,v1_index)),0) + delta

    # Separate the keys (row, col) and values of the dictionary
    rows, cols = zip(*laplace_dict.keys())
    values = list(laplace_dict.values())

    # Create a COO matrix from the non-zero elements
    coo = coo_matrix((values, (rows, cols)), shape=(mesh.n_vertices(), mesh.n_vertices()))

    # Convert the COO matrix to a CSR matrix
    csr = coo.tocsr()
    return csr

laplace_csr = generate_Laplace_matrix(mesh)
# print(laplace_csr)
print(laplace_csr.toarray())
print(laplace_csr.toarray() == laplace_matrix)

[[-3.9   1.35  1.35  0.    0.    0.    1.2   0.  ]
 [ 1.35 -3.9   0.    1.35  0.    0.    0.    1.2 ]
 [ 1.35  0.   -3.9   1.35  1.2   0.    0.    0.  ]
 [ 0.    1.35  1.35 -3.9   0.    1.2   0.    0.  ]
 [ 0.    0.    1.2   0.   -3.9   1.35  1.35  0.  ]
 [ 0.    0.    0.    1.2   1.35 -3.9   0.    1.35]
 [ 1.2   0.    0.    0.    1.35  0.   -3.9   1.35]
 [ 0.    1.2   0.    0.    0.    1.35  1.35 -3.9 ]]
[[ True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True]]


In [22]:
mesh_pl = os.path.join('..', 'models', 'armadillo.obj')
tmesh= trimesh.load(mesh_pl)
# mesh = read_mesh_om('cube.obj')


In [33]:
def trimesh_Generate_Area_List(tmesh):
    area_list = np.zeros((len(tmesh.vertices)))
    face_count = tmesh.faces.shape[0]
    face_area = np.asarray(tmesh.area_faces)
    faces = np.asarray(tmesh.faces)
    for i in tqdm(range(face_count)):
        area_list[faces[i]] += face_area[i] / 3

    return area_list
# print(area_list)


def trimesh_Generate_Laplace_matrix(tmesh):
    vertices_face_indexs = [[0,1,2],[1,0,2],[2,0,1]] 
    laplace_dict = {}
    area_list = trimesh_Generate_Area_List(tmesh)
    face_angles = np.asarray(tmesh.face_angles)
    faces = np.asarray(tmesh.faces)
    for face,angles in tqdm(zip(faces, face_angles)):
        for i in range(3):
            current_angle = angles[i]
            v0_index,v1_index, v2_index= face[vertices_face_indexs[i]]
            delta = 0.5 / area_list[v0_index] / np.tan(current_angle)
            if delta > 0.000001:

                laplace_dict[(v1_index,v1_index)] = laplace_dict.get((v1_index,v1_index),0) - delta
                laplace_dict[(v2_index,v2_index)] = laplace_dict.get((v2_index,v2_index),0) - delta

                laplace_dict[(v1_index,v2_index)] = laplace_dict.get((v1_index,v2_index),0) + delta
                laplace_dict[(v2_index,v1_index)] = laplace_dict.get((v2_index,v1_index),0) + delta

    # Separate the keys (row, col) and values of the dictionary
    rows, cols = zip(*laplace_dict.keys())
    values = list(laplace_dict.values())

    # Create a COO matrix from the non-zero elements
    # print(tmesh.vertices)
    coo = coo_matrix((values, (rows, cols)), shape=(len(tmesh.vertices), len(tmesh.vertices)))

    # Convert the COO matrix to a CSR matrix
    csr = coo.tocsr()
    return csr

laplace_csr = trimesh_Generate_Laplace_matrix(tmesh)
# print(laplace_csr)
# print(laplace_csr.toarray())
# print(laplace_csr.toarray() == laplace_matrix)

100%|██████████| 50382/50382 [00:00<00:00, 618911.49it/s]
50382it [00:00, 79542.19it/s]


In [ ]:
rtx = np.array([3,4,5])
rtx +1


array([4, 5, 6])

In [ ]:
# Trimesh Version, to be faster!

mesh_pl = os.path.join('cube.obj')
mesh = trimesh.load(mesh_pl)
# mesh = read_mesh_om('cube.obj')

def generate_Area_List(mesh):
    
    area_list = np.zeros((mesh.n_vertices()))
    for f_it in mesh.faces():
    # help(f_it)
        vertices_index, vertices = get_Vertices_From_Face(mesh,f_it)
        current_area = calculate_Triangle_Area(vertices[0],vertices[1],vertices[2])
        # print(current_area);
        area_list[vertices_index] += current_area/3
    return area_list


def generate_Laplace_matrix(mesh):
    vertices_face_indexs = [[0,1,2],[1,0,2],[2,0,1]] 
    laplace_dict = {}
    area_list = generate_Area_List(mesh)
    for f_it in mesh.faces():
        vertices_index, vertices = get_Vertices_From_Face(mesh,f_it)
        for current_indexs in vertices_face_indexs:
            
            v0, v1, v2 = vertices[current_indexs]
            v0_index, v1_index,v2_index = vertices_index[current_indexs]

            current_angle = calculate_Triangle_Angle(v0, v1, v2)

            delta = 0.5 / area_list[v0_index] / np.tan(current_angle)
            if delta > 0.000001:

                laplace_dict[(v1_index,v1_index)] = laplace_dict.get(((v1_index,v1_index)),0) - delta
                laplace_dict[(v2_index,v2_index)] = laplace_dict.get(((v2_index,v2_index)),0) - delta

                laplace_dict[(v1_index,v2_index)] = laplace_dict.get(((v1_index,v2_index)),0) + delta
                laplace_dict[(v2_index,v1_index)] = laplace_dict.get(((v2_index,v1_index)),0) + delta

    # Separate the keys (row, col) and values of the dictionary
    rows, cols = zip(*laplace_dict.keys())
    values = list(laplace_dict.values())

    # Create a COO matrix from the non-zero elements
    coo = coo_matrix((values, (rows, cols)), shape=(mesh.n_vertices(), mesh.n_vertices()))

    # Convert the COO matrix to a CSR matrix
    csr = coo.tocsr()
    return csr

laplace_csr = generate_Laplace_matrix(mesh)
# print(laplace_csr)
print(laplace_csr.toarray())
print(laplace_csr.toarray() == laplace_matrix)

In [ ]:
# # Your random element generation function
# value = 5
# def calculate_element(row, col):
#     # Your logic to compute the element goes here
#     return value

# # Initialize an empty dictionary for the sparse matrix
# sparse_matrix_dict = {}

# # Set the dimensions of your matrix
# num_rows = 1000
# num_columns = 1000

# # Define the number of non-zero elements to generate
# num_elements = 10000

# # Calculate the non-zero elements and add them to the dictionary
# for _ in range(num_elements):
#     row = np.random.randint(0, num_rows)
#     col = np.random.randint(0, num_columns)
#     value = calculate_element(row, col)

#     # Add the element to the dictionary with (row, col) as key
#     if value != 0:
#         sparse_matrix_dict[(row, col)] = value

# # Separate the keys (row, col) and values of the dictionary
# rows, cols = zip(*sparse_matrix_dict.keys())
# values = list(sparse_matrix_dict.values())

# # Create a COO matrix from the non-zero elements
# coo = coo_matrix((values, (rows, cols)), shape=(num_rows, num_columns))

# # Convert the COO matrix to a CSR matrix
# csr = coo.tocsr()

# print(csr)

  (0, 16)	5
  (0, 44)	5
  (0, 71)	5
  (0, 217)	5
  (0, 512)	5
  (0, 586)	5
  (0, 666)	5
  (0, 761)	5
  (0, 779)	5
  (0, 823)	5
  (0, 838)	5
  (1, 41)	5
  (1, 82)	5
  (1, 147)	5
  (1, 291)	5
  (1, 375)	5
  (1, 602)	5
  (1, 726)	5
  (1, 880)	5
  (1, 936)	5
  (1, 997)	5
  (2, 82)	5
  (2, 651)	5
  (2, 684)	5
  (2, 690)	5
  :	:
  (997, 601)	5
  (997, 866)	5
  (997, 919)	5
  (998, 31)	5
  (998, 242)	5
  (998, 264)	5
  (998, 276)	5
  (998, 328)	5
  (998, 514)	5
  (998, 581)	5
  (998, 582)	5
  (998, 650)	5
  (999, 99)	5
  (999, 194)	5
  (999, 314)	5
  (999, 365)	5
  (999, 441)	5
  (999, 452)	5
  (999, 539)	5
  (999, 562)	5
  (999, 587)	5
  (999, 617)	5
  (999, 688)	5
  (999, 824)	5
  (999, 911)	5
